## Imports

In [0]:
!pip install imbalanced-learn

In [0]:
!pip install boruta

In [0]:
!pip install sklearn-genetic

In [0]:
!pip install hyperopt

In [0]:
!pip install xgboost

In [0]:
!pip install lightgbm

In [0]:
!pip install implicit

In [0]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType, StringType, DateType, DoubleType

import numpy as np
import pandas as pd
from IPython.display import Image
import warnings
from sklearn.feature_selection import f_classif, mutual_info_classif, SequentialFeatureSelector, SelectKBest
from scipy.stats import chi2_contingency
from sklearn.base import BaseEstimator, TransformerMixin, clone
from sklearn.pipeline import FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer, MissingIndicator
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, recall_score, precision_score, make_scorer, f1_score, accuracy_score
from sklearn.model_selection import cross_val_predict, LeaveOneOut, cross_val_score, StratifiedKFold, GridSearchCV
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import RandomOverSampler, SMOTE
from boruta import BorutaPy
from genetic_selection import GeneticSelectionCV
from hyperopt import fmin, tpe, hp, Trials
from hyperopt.pyll.base import scope
from hyperopt.pyll.stochastic import sample
from functools import partial
import xgboost as xgb
from lightgbm import LGBMClassifier
from scipy.sparse import coo_matrix
import implicit
from implicit.evaluation import ndcg_at_k

In [0]:
spark = SparkSession.builder.appName("ifood_case").getOrCreate()

In [0]:
path_features = 'dbfs:/FileStore/ifood_case/data/processed/df_customers_offers.parquet'

## Extract

In [0]:
df_features = spark.read.parquet(path_features)
df_features.groupBy('offer_accepted').count().display()

offer_accepted,count
0,45856
1,24000


In [0]:
df = df_features.toPandas()

In [0]:
df.head()

,account_id,split,amount,offer_id,min_value,duration,discount_value,avg_ticket,variability_ticket,offer_acceptance_rate,...,is_female,is_other,credit_card_limit,offer_accepted,is_bogo_offer,is_discount_offer,is_informational_offer,has_web_notification,has_mobile_notification,has_social_notification
0,2c4f8a7e0933415ca5ceda66946c1255,train,0.0,5a8bc65990b245e5a138643cd4eb9837,0,3.0,0,19.543333,1.944154,0.250000,...,0,0,79000.0,0,0,0,1,0,1,1
1,27586221a0a5482b960ac41506d2c316,train,0.0,4d5c57ea9a6940dd891ad53e9dbe8da0,10,5.0,10,27.118000,8.582239,0.285714,...,0,0,114000.0,0,1,0,0,1,1,1
2,36399e0a45554cafabaab01cdb80c480,validation,0.0,0b1e1539f2cc45b7b9fa7c272da2e1d7,20,10.0,5,28.390000,2.941751,0.333333,...,1,0,84000.0,0,0,1,0,1,0,0
3,aeea18cf2e8d455c98453c546292a9f6,train,0.0,4d5c57ea9a6940dd891ad53e9dbe8da0,10,5.0,10,22.278571,6.869938,0.428571,...,1,0,73000.0,0,1,0,0,1,1,1
4,5fc96150a4994e3c982e723d06d35e8b,train,0.0,5a8bc65990b245e5a138643cd4eb9837,0,3.0,0,24.080000,4.299209,0.285714,...,1,0,81000.0,0,0,0,1,0,1,1


## ML Tests

### Feature Transformers

In [0]:
target = 'offer_accepted'
key_feats = ['account_id', 'offer_id', 'split', 'amount']
feats = df_features_pd.drop([target] + key_feats, axis=1).columns
num_feats = [feat for feat in feats if df_features_pd[feat].dtype != 'O']
cat_feats = [feat for feat in feats if feat not in num_feats]

In [0]:
num_transformer = FeatureUnion(    
    [
        ('num_pipe', Pipeline(
            [
                ('norm', StandardScaler()),
                ('nan_input', SimpleImputer())
            ]
        )),
        ('nan_flag', MissingIndicator(error_on_new=False))
    ]
)
feat_transformer = ColumnTransformer(
    [
        ('num_trans', num_transformer, num_feats),
        ('cat_trans', OneHotEncoder(handle_unknown='ignore'), cat_feats)    
    ],
    remainder='passthrough', sparse_threshold=0
)

In [0]:
df_train = df[df['split'] == 'train']
X_train = df_train[feats]
y_train = df_train["offer_accepted"]

df_test = df[df['split'] != 'train']
X_test = df_test[feats]
y_test = df_test["offer_accepted"]

df_validation = df[df['split'] == 'validation']
X_validation = df_validation[feats]
y_validation = df_validation["offer_accepted"]


### Models

In [0]:
def optimize_hyperparameters(opt_space, pipe, X, y, max_evals=100):

    def obj(x):
        
        model = clone(pipe).set_params(**x)
        preds = cross_val_predict(model, X, y, cv=3, n_jobs=-1)

        return -f1_score(y, preds, average='macro')


    best_hypers = fmin(obj, space=opt_space, algo=tpe.suggest, 
                    max_evals=max_evals, return_argmin=False)

    return best_hypers

def optimize_als_hyperparameters(opt_space, train_matrix, val_matrix, max_evals=50):
    def obj(params):
        model = implicit.als.AlternatingLeastSquares(
            factors=params['factors'],
            regularization=params['regularization'],
            iterations=params['iterations'],
            random_state=42
        )
        model.fit((train_matrix * params['alpha']).astype('double'))

        score = ndcg_at_k(model, train_matrix, val_matrix, K=10)
        return -score 

    best_hypers = fmin(obj, space=opt_space, algo=tpe.suggest, max_evals=max_evals, return_argmin=False)
    return best_hypers

### Algorithms Comparison

In [0]:
lr_pipe = Pipeline(
  [
      ('feat_trans', feat_transformer),
      ('over', SMOTE()),
      ('logreg', LogisticRegression(random_state = 0))
]
)

# lr_opt_space = {'logreg__solver': hp.choice('logreg__solver', ['liblinear', 'lbfgs']),
#                 'logreg__C': hp.loguniform('logreg__C', np.log(1e-5), np.log(100))}
               
lr_opt_space = {'logreg__warm_start' : hp.choice('logreg__warm_start', [True, False]),
                'logreg__fit_intercept' : hp.choice('logreg__fit_intercept', [True, False]),
                'logreg__tol' : hp.uniform('logreg__tol', 0.00001, 0.0001),
                'logreg__C' : hp.uniform('logreg__C', 0.05, 3),
                'logreg__solver' : hp.choice('logreg__solver', ['newton-cg', 'lbfgs', 'liblinear']),
                'logreg__multi_class' : 'auto',
                'logreg__class_weight' : 'balanced'}

best_hypers = optimize_hyperparameters(lr_opt_space, clone(lr_pipe), X_train, y_train)
model = clone(lr_pipe).set_params(**best_hypers).fit(X_train, y_train)
y_pred = cross_val_predict(clone(lr_pipe).set_params(**best_hypers), X_test, y_test, cv=3)
y_pred_prob = cross_val_predict(clone(lr_pipe).set_params(**best_hypers), X_test, y_test, cv=3, method='predict_proba')

print(classification_report(y_test, y_pred))

100%|██████████| 100/100 [10:41<00:00,  6.42s/trial, best loss: -0.5448537140330508]
              precision    recall  f1-score   support

           0       0.64      0.69      0.66      9341
           1       0.70      0.65      0.67     10390

    accuracy                           0.67     19731
   macro avg       0.67      0.67      0.67     19731
weighted avg       0.67      0.67      0.67     19731



In [0]:
y_pred

Out[56]: array([0, 0, 0, ..., 0, 0, 0])

In [0]:
xgb_pipe = Pipeline(
  [
      ('feat_trans', feat_transformer),
      ('over', SMOTE()),
      ('xgb', xgb.XGBClassifier())
]
)

xgb_opt_space = {'xgb__max_depth':  hp.choice('xgb__max_depth', np.arange(1, 14, dtype=int)),
                'xgb__gamma': hp.uniform ('xgb__gamma', 1,9),
                'xgb__reg_alpha' : hp.quniform('xgb__reg_alpha', 40,180,1),
                'xgb__reg_lambda' : hp.uniform('xgb__reg_lambda', 0,1),
                'xgb__colsample_bytree' : hp.uniform('xgb__colsample_bytree', 0.5,1),
                'xgb__min_child_weight' : hp.quniform('xgb__min_child_weight', 0, 10, 1),
                'xgb__n_estimators': 180}

best_hypers = optimize_hyperparameters(xgb_opt_space, clone(xgb_pipe), X_train, y_train)
model = clone(xgb_pipe).set_params(**best_hypers).fit(X_train, y_train)
y_pred = cross_val_predict(clone(xgb_pipe).set_params(**best_hypers), X_test, y_test, cv=3)
y_pred_prob = cross_val_predict(clone(xgb_pipe).set_params(**best_hypers), X_test, y_test, cv=3, method='predict_proba')

print(classification_report(y_test, y_pred))

100%|██████████| 100/100 [11:47<00:00,  7.07s/trial, best loss: -0.5805163075416316]
              precision    recall  f1-score   support

           0       0.58      0.93      0.72      9341
           1       0.86      0.41      0.55     10390

    accuracy                           0.65     19731
   macro avg       0.72      0.67      0.63     19731
weighted avg       0.73      0.65      0.63     19731

